In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
import time

In [2]:
def akbo_crawler():
    df=pd.DataFrame()
    for i in range(1, 101):
        ajax_url ='http://www.akbobada.com/ajaxPartgList.html?pageIndex={}&order1=&order2=&order3=&catCode=02&groupCode=12&searchText='
        r_ajax = requests.get(ajax_url.format(i))
        r_ajax.text[1:-1]
        json_result = r_ajax.json()
        df=df.append(pd.DataFrame(json_result))
    df=df.set_index('AKBO_ID')

    df['파트']=''
    df['설명']=''
    df['key(조)']=''
    df['페이지']=''

    del df['ALBUM_ID']
    del df['PART_CLASS_DISP']
    del df['ak_lyric']
    del df['pageCounts']
    del df['sample_path']

    mylist=df.index.values

    for n in range(len(mylist)):
        url='http://www.akbobada.com/musicDetail.html?musicID={}&p=1'
        r=requests.get(url.format(mylist[n]))
        r.text
        soup=bs(r.text, 'lxml')
        list2=soup.findAll('tr')
        k=0    
        while True:

            if len(soup.findAll('tr')[k].findAll('td')) !=0 and '피아노' in re.sub('\s+', '', [value.text for value in soup.findAll('tr')[k].findAll('td')][0]):
                list1=[]
                for a in range(len([value.text for value in soup.findAll('tr')[k].findAll('td')])):
                    list1.append(re.sub('\s+', '', [value.text for value in soup.findAll('tr')[k].findAll('td')][a]))

                df.at[mylist[n],'파트']=list1[0]
                df.at[mylist[n],'설명']=list1[1]
                df.at[mylist[n],'key(조)']=list1[2]
                df.at[mylist[n],'페이지']=list1[3]
                break
            else:
                k=k+1
    
    df.to_csv('akbo.csv')


In [4]:
start_time = time.time()
akbo_crawler()
print("--- %s seconds ---" % (time.time() - start_time))

--- 874.9229295253754 seconds ---
